In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')


import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

PATH = '../'
RND_SEED = 44

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 21.4 s


In [3]:
train[train['sat_id'] == 0].head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01T00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1,2014-01-01T00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2,2014-01-01T01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,3,2014-01-01T02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,4,2014-01-01T03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [4]:
train.groupby('sat_id').apply(lambda x: np.linalg.norm(x['x'] - x['x'].shift(1).fillna(0)))

sat_id
0       81648.480368
1      183541.882391
2      232012.517213
3      245649.819107
4      191856.438222
5      163938.902355
6      221734.036473
7      255255.505317
8      170149.097052
9      226233.664627
10     180480.849601
11     179163.308398
12     230857.728576
13     200762.257237
14     198751.495171
15      57215.309266
16     268620.218738
17      58398.572869
18     157096.965145
19     226509.817673
20     171728.014259
21     161661.526163
22     160557.316121
23     220801.645724
24     234649.003511
25     198550.956250
26     199365.432655
27     186898.890231
28     220666.857918
29     239454.001293
           ...      
570    163892.369974
571    171910.842346
572    231545.327843
573    217025.134472
574     58457.556155
575    199141.697487
576    193315.922399
577    212398.233347
578    105377.082367
579    178131.693611
580    229119.176302
581    199225.806668
582    136962.822153
583    140517.617916
584    196253.717951
585    222100.229028
586   

In [18]:
satData = pd.DataFrame({'sat_id':train['sat_id'].unique(), 'counts': train.groupby('sat_id').count().iloc[:, 0]})
for col in train.columns[9:]:
    satData.loc[:, col+'_std'] = train.groupby('sat_id')[col].std()
for col in train.columns[9:]:
    satData.loc[:, col+'_norm'] = train.groupby('sat_id').apply(lambda x: np.linalg.norm(x[col] - x[col].shift(10).fillna(0)))

In [19]:
linearSat =[5,
 6,
 8,
 9,
 11,
 14,
 20,
 22,
 24,
 26,
 28,
 29,
 30,
 31,
 34,
 35,
 36,
 39,
 40,
 42,
 43,
 44,
 46,
 47,
 49,
 51,
 53,
 54,
 55,
 57,
 58,
 60,
 61,
 64,
 67,
 68,
 69,
 71,
 72,
 74,
 75,
 76,
 79,
 81,
 83,
 91,
 93,
 97,
 102,
 103,
 104,
 106,
 107,
 108,
 111,
 113,
 114,
 115,
 116,
 119,
 120,
 121,
 122,
 128,
 129,
 130,
 131,
 132,
 135,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 152,
 154,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 168,
 170,
 171,
 172,
 176,
 179,
 180,
 183,
 187,
 191,
 193,
 196,
 198,
 200,
 203,
 207,
 210,
 211,
 212,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 224,
 225,
 226,
 227,
 229,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 240,
 245,
 250,
 252,
 253,
 254,
 259,
 260,
 261,
 262,
 263,
 265,
 268,
 271,
 273,
 276,
 277,
 279,
 281,
 284,
 285,
 287,
 289,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 305,
 306,
 307,
 309,
 310,
 311,
 312,
 313,
 314,
 316,
 317,
 319,
 321,
 322,
 324,
 326,
 330,
 331,
 332,
 333,
 335,
 337,
 339,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 350,
 352,
 353,
 355,
 356,
 358,
 360,
 362,
 367,
 369,
 370,
 373,
 374,
 377,
 378,
 379,
 383,
 384,
 386,
 387,
 388,
 389,
 391,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 403,
 404,
 405,
 408,
 410,
 411,
 413,
 414,
 417,
 419,
 423,
 424,
 425,
 426,
 428,
 430,
 431,
 432,
 433,
 435,
 436,
 438,
 442,
 443,
 444,
 446,
 447,
 448,
 455,
 456,
 458,
 459,
 460,
 462,
 463,
 465,
 466,
 467,
 469,
 471,
 474,
 479,
 485,
 487,
 488,
 489,
 492,
 496,
 499,
 504,
 507,
 510,
 515,
 516,
 517,
 519,
 520,
 522,
 524,
 525,
 526,
 528,
 532,
 534,
 537,
 540,
 543,
 544,
 545,
 547,
 553,
 556,
 557,
 562,
 563,
 564,
 566,
 567,
 570,
 571,
 572,
 573,
 574,
 578,
 579,
 582,
 583,
 586,
 589,
 591,
 593,
 594,
 599]

In [20]:
target = np.zeros(satData.shape[0])
target[linearSat] = 1

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

model = RandomForestClassifier()
trainSize = 0.8
size = int(satData.shape[0] * trainSize)
Xtr, Xval = satData.iloc[:size, 1:], satData.iloc[size:, 1:]
ytr, yval = target[:size], target[size:]

model.fit(Xtr, ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
from sklearn.metrics import accuracy_score, roc_auc_score
roc_auc_score(yval, model.predict(Xval))

0.5353793691389599